In [1]:
# Pandas is used for data manipulation
import pandas as pd
import numpy as np

# Read in data and display first 5 rows
z_data = pd.read_csv('data_task_groups_roi_deact.csv',sep=';',decimal='.')
z_data.drop(list(z_data.filter(regex = '_act')),axis = 1, inplace = True)
data = z_data

#we need the list with the labels to test the predictions; everytime the label or proband changes, we add it to the list
labelsComputingAccuracy = []

condition = z_data.at[0, 'task']
participant = z_data.at[0, 'proband']
    
for i in range(len(z_data)):
    if condition != z_data.at[i, 'task'] or participant != z_data.at[i, 'proband']:
        labelsComputingAccuracy.append(condition)
        condition = z_data.at[i, 'task']
        participant = z_data.at[i, 'proband']
labelsComputingAccuracy.append(condition)

# recode word labels to num labels so t_pot can handle the lables =======
labels = []
for i in range(len(z_data)):
    if z_data.at[i,'task'] == 'C':
        labels.append(0)
    if z_data.at[i,'task'] == 'R':
        labels.append(1)
    if z_data.at[i,'task'] == 'S':
        labels.append(2)

#=== remove everything that is not a feature ===================================================
z_data = z_data.drop('proband', axis = 1)
z_data = z_data.drop('scan', axis = 1)
z_data = z_data.drop('trial', axis = 1)
z_data = z_data.drop('task', axis = 1)
z_data = z_data.drop('snippet', axis = 1)
z_data = z_data.drop('response', axis = 1)

z_data.drop(list(z_data.filter(regex = 'aggr')),axis = 1, inplace = True)
#=== creating training and validation set ======================================================
# Saving feature names for later use
feature_list = z_data.columns

# Convert to numpy array
features = np.array(z_data)
labels = np.array(labels)

#split the data into train and validation set
#get the rows excluding the last indexed (i.e., row with the index 10973 is the last to include)
#the rows until 10973 contain the first 12 participants, the remaining rows the last 4
training_features = features[0:10974]
training_features = np.array(training_features)
testing_features = features[10974:14634]
testing_features = np.array(testing_features)

training_target = labels[0:10974]
training_target = np.array(training_target)
testing_target = labels[10974:14634]
testing_target = np.array(testing_target)



In [34]:
z_data.head(5)

,149-108-79,149-111-79,149-108-82,152-108-70,152-111-70,152-108-73,152-111-73,152-105-76,152-108-76,152-105-79,...,194-165-130,194-168-130,194-171-130,194-174-130,194-165-133,194-168-133,194-171-133,194-174-133,194-171-136,194-174-136
0,0.527377,0.986905,0.437735,0.741219,1.127029,1.240144,1.270220,0.496916,0.840957,0.357522,...,1.177977,0.714734,0.157226,-1.142369,0.817561,0.340247,0.282420,0.236172,0.331725,0.911182
1,-0.074035,-0.051551,0.428573,-1.585273,-1.724940,-1.273733,-1.495270,-0.393721,-0.724471,0.482341,...,0.200067,0.252834,-0.330978,-0.927583,0.257427,0.259700,0.001006,-0.083581,0.180125,-0.073254
2,0.655263,0.268250,0.213250,0.150772,0.413103,0.960824,0.960234,0.483673,0.910532,-0.016937,...,0.955328,0.707840,0.178001,-0.599086,0.705534,0.493286,0.111726,0.262597,-0.421681,0.153482
3,-0.426586,-0.766612,-0.180745,-0.744527,-0.652181,-0.652022,-0.460824,-0.724812,-0.427198,-0.391395,...,1.195439,0.558470,0.453265,-0.150561,0.065963,-0.171226,0.243207,0.812254,-0.244814,0.629914
4,-1.740014,-1.075633,-0.455625,-1.184153,-0.065341,-1.829369,-0.662837,-2.896774,-2.912908,-1.167617,...,-0.996126,-0.135530,-0.188153,-1.451914,-1.335390,-1.077380,-0.720983,-0.564530,-0.851214,0.098950


In [ ]:
from tpot import TPOTClassifier
fileNameTPot = 'Deact/tpot_mnist_pipeline_NotAggregatedParticipantSplit_Act'

tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2,n_jobs = 20)
tpot.fit(training_features, training_target)
print(tpot.score(testing_features, testing_target))
tpot.export(fileNameTPot + '.py')

C:\Users\author\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap_external.py:426: ImportWarning: Not importing directory C:\Users\author\Anaconda3\lib\site-packages\mpl_toolkits: missing __init__
  _warnin

In [ ]:
with open(fileNameTPot + '.py') as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
# content = [x.strip() for x in content] 

#      or 'exported_pipeline = ' not in line
cleanedContent = []
for line in content:
    if 'tpot_data' not in line and 'training_target, testing_target' not in line:
        cleanedContent.append(line)

fileForLearning = fileNameTPot + '_cleaned.py'
with open(fileForLearning, 'w') as filehandle:  
    for line in cleanedContent:
        filehandle.write('%s\n' % line)

In [5]:
%run -i $fileForLearning

In [6]:
z_data = data
condition = z_data.at[10974, 'task']
participant = z_data.at[10974, 'proband']
majorityVote = []
labelsForAcc = []
countComprehension = 0
countRest = 0
countSyntax = 0
    
for i,j in zip(range(10974,len(z_data)),range(len(results))):
    if condition != z_data.at[i, 'task'] or participant != z_data.at[i, 'proband']:
        labelsForAcc.append(condition)
        condition = z_data.at[i, 'task']
        participant = z_data.at[i, 'proband']
        if (countComprehension > countRest) and (countComprehension > countSyntax):
            majorityVote.append('C')
        elif countSyntax > countRest and countSyntax > countComprehension:
                majorityVote.append('S')
        elif countRest > countComprehension and countRest > countSyntax:
            majorityVote.append('R')
        elif countComprehension == countRest:
            majorityVote.append('CR')
        elif countComprehension == countSyntax:
            majorityVote.append('CS')
        else:
            majorityVote.append('SR')
        countComprehension = 0
        countRest = 0
        countSyntax = 0

    if (results[j] == 0):
        countComprehension = countComprehension + 1
    if (results[j] == 1):
        countRest = countRest + 1
    if (results[j] == 2):
        countSyntax = countSyntax + 1

labelsForAcc.append(condition)
if (results[j] == 0):
    countComprehension = countComprehension + 1
if (results[j] == 1):
    countRest = countRest + 1
if (results[j] == 2):
    countSyntax = countSyntax + 1

if (countComprehension > countRest) and (countComprehension > countSyntax):
    majorityVote.append('C')
elif countSyntax > countRest and countSyntax > countComprehension:
        majorityVote.append('S')
elif countRest > countComprehension and countRest > countSyntax:
    majorityVote.append('R')
elif countComprehension == countRest:
    majorityVote.append('CR')
elif countComprehension == countSyntax:
    majorityVote.append('CS')
else:
    majorityVote.append('SR')


In [7]:
num_matches = 0;
for a, b in zip(majorityVote, labelsForAcc):
    if b in a:
        num_matches = num_matches + 1
print('Number of matches:',num_matches,'(of',len(labelsForAcc),')')

accuracy = num_matches/len(labelsForAcc)*100
print('Accuary: ',accuracy)


Number of matches: 117 (of 196 )
Accuary:  59.693877551020414


In [ ]:
with open('listOfAccuracies.txt','a+') as f:
    f.write('%s\n' % 'NoAgg_Deact' + str(accuracy) + '\n')
